In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Scrape match summary data
years = ['2020-2020', '2021-2021', '2022-2022', '2024-2024']
base_url = "https://www.espncricinfo.com/records/year/team-match-results/{}/twenty20-internationals-3?team=6"

all_rows = []
all_headers = None

for yr in years:
    url = base_url.format(yr)
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('div', class_='ds-p-0')

    # Extract headers
    if all_headers is None:
        all_headers = [th.text.strip() for th in table.find('thead').find_all('td')] + ['Scorecard Link']
    
    # Extract rows for df
    for tr in table.find('tbody').find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        scorecard_td = tr.find_all('td')[-1]
        scorecard_link = scorecard_td.find('a')['href'] if scorecard_td.find('a') else None
        full_scorecard_link = "https://www.espncricinfo.com" + scorecard_link if scorecard_link else None
        cells.append(full_scorecard_link)
        all_rows.append(cells)

# Create match summary df
match_summary_df = pd.DataFrame(all_rows, columns=all_headers)

In [3]:
match_summary_df

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard,Scorecard Link
0,Australia,India,Australia,12 runs,Sydney,"Dec 8, 2020",T20I # 1116,https://www.espncricinfo.com/series/india-in-a...
1,Australia,India,India,6 wickets,Sydney,"Dec 6, 2020",T20I # 1115,https://www.espncricinfo.com/series/india-in-a...
2,Australia,India,India,11 runs,Canberra,"Dec 4, 2020",T20I # 1114,https://www.espncricinfo.com/series/india-in-a...
3,New Zealand,India,India,7 runs,Mount Maunganui,"Feb 2, 2020",T20I # 1037,https://www.espncricinfo.com/series/india-in-n...
4,New Zealand,India,tied,-,Wellington,"Jan 31, 2020",T20I # 1036,https://www.espncricinfo.com/series/india-in-n...
...,...,...,...,...,...,...,...,...
76,India,Pakistan,India,6 runs,New York,"Jun 9, 2024",T20I # 2658,https://www.espncricinfo.com/series/icc-men-s-...
77,India,Ireland,India,8 wickets,New York,"Jun 5, 2024",T20I # 2639,https://www.espncricinfo.com/series/icc-men-s-...
78,India,Afghanistan,tied,-,Bengaluru,"Jan 17, 2024",T20I # 2435,https://www.espncricinfo.com/series/afghanista...
79,India,Afghanistan,India,6 wickets,Indore,"Jan 14, 2024",T20I # 2431,https://www.espncricinfo.com/series/afghanista...


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_ground_details(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the request was unsuccessful
        soup = BeautifulSoup(response.content, 'html.parser')

        #tables with the specific class
        tables = soup.find_all('table', class_='ds-w-full ds-table ds-table-sm ds-table-auto')

        target_table = None

        # Identify the target table by looking for specific content (beacuse there are multiple tables with this calss)
        for table in tables:
            if table.find('td', string='Toss') or table.find('td', string='Series'):
                target_table = table
                break

        # Extract data from the target table
        if target_table is not None:
            rows = []
            for row in target_table.find_all('tr'):
                cells = row.find_all(['td', 'th'])
                if len(cells) == 2:  # Ensuring it has a header and value pair
                    field = cells[0].text.strip()
                    value = cells[1].text.strip()
                    rows.append([field, value])

            # Convert to DataFrame
            df = pd.DataFrame(rows, columns=['Field', 'Value'])

            # Pivot the DataFrame to have fields as columns
            pivot_df = df.set_index('Field').T
            pivot_df['URL'] = url  # Add a column for the URL to identify the data source

            return pivot_df
        else:
            print("Target table not found for URL:", url)
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return pd.DataFrame()

# foR EVERY SCORECARD FROM MATCHES DF SCRAPE THE DATA

all_data = []

for _, row in match_summary_df.iterrows():
    url = row['Scorecard Link']
    details_df = scrape_ground_details(url)
    if not details_df.empty:
        all_data.append(details_df)

# Combine all individual DataFrames into one
final_df = pd.concat(all_data, ignore_index=True)

In [5]:
final_df

Field,Toss,Series,Season,Player Of The Match,Player Of The Series,Series result,Match number,Hours of play (local time),Match days,Umpires,TV Umpire,Reserve Umpire,Match Referee,URL,T20I debut,IND Player Replacement,Points
0,"India, elected to field first",India tour of Australia,2020/21,Mitchell Swepson,Hardik Pandya,India won the 3-match series 2-1,T20I no. 1116,"19.10 start, First Session 19.10-20.40, Interv...",08 December 2020 - night (20-over match),Gerard AboodRod Tucker,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,NaN,NaN,NaN
1,"India, elected to field first",India tour of Australia,2020/21,Hardik Pandya,NaN,India led the 3-match series 2-0,T20I no. 1115,"19.10 start, First Session 19.10-20.40, Interv...",06 December 2020 - night (20-over match),Paul WilsonSam Nogajski,Rod Tucker,Gerard Abood,David Boon,https://www.espncricinfo.com/series/india-in-a...,Daniel Sams,NaN,NaN
2,"Australia, elected to field first",India tour of Australia,2020/21,Yuzvendra Chahal,NaN,India led the 3-match series 1-0,T20I no. 1114,"19.10 start, First Session 19.10-20.40, Interv...",04 December 2020 - night (20-over match),Rod TuckerShawn Craig,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,T Natarajan,"Substitute: Yuzvendra Chahal in, Ravindra Jade...",NaN
3,"India, elected to bat first",India tour of New Zealand,2019/20,Jasprit Bumrah,KL Rahul,India won the 5-match series 5-0,T20I no. 1037,"20.00 start, First Session 20.00-21.30, Interv...",2 February 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Damian Morrow,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
4,"New Zealand, elected to field first",India tour of New Zealand,2019/20,Shardul Thakur,NaN,India led the 5-match series 4-0,T20I no. 1036,"20.00 start, First Session 20.00-21.30, Interv...",31 January 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Garth Stirrat,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Pakistan, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2658,NaN,9 June 2024 - day (20-over match),Richard IllingworthDRSRod TuckerDRS,Chris Gaffaney,Sharfuddoula,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Pakistan 0"
77,"India, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2639,NaN,5 June 2024 - day (20-over match),Alex WharfDRSChris GaffaneyDRS,Richard Kettleborough,Chris Brown,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Ireland 0"
78,"India, elected to bat first",Afghanistan tour of India,2023/24,Rohit Sharma,Shivam Dube,India won the 3-match series 3-0,T20I no. 2435,"19.00 start, First Session 19.00-20.30, Interv...",17 January 2024 - night (20-over match),Jayaraman MadanagopalDRSVirender SharmaDRS,Rohan Pandit,Mohit Krishnadas,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,Mohammad Saleem,NaN,NaN
79,"India, elected to field first",Afghanistan tour of India,2023/24,Axar Patel,NaN,India led the 3-match series 2-0,T20I no. 2431,"19.00 start, First Session 19.00-20.30, Interv...",14 January 2024 - night (20-over match),Rohan PanditDRSVirender SharmaDRS,Jayaraman Madanagopal,Saiyed Khalid,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,NaN,NaN,NaN


In [ ]:
final_df.to_csv("match_details")

In [35]:
MatchDetails = pd.read_excel("C:\\Users\\Harshita\\Documents\\BigDataFiles\\CSV_files\\matchDetails.xlsx")

In [36]:
MatchDetails

,Toss,Series,Season,Player Of The Match,Player Of The Series,Series result,Match number,Hours of play (local time),Match days,Umpires,TV Umpire,Reserve Umpire,Match Referee,URL,T20I debut,IND Player Replacement,Points
0,"India, elected to field first",India tour of Australia,2020/21,Mitchell Swepson,Hardik Pandya,India won the 3-match series 2-1,T20I no. 1116,"19.10 start, First Session 19.10-20.40, Interv...",08 December 2020 - night (20-over match),Gerard AboodRod Tucker,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,NaN,NaN,NaN
1,"India, elected to field first",India tour of Australia,2020/21,Hardik Pandya,NaN,India led the 3-match series 2-0,T20I no. 1115,"19.10 start, First Session 19.10-20.40, Interv...",06 December 2020 - night (20-over match),Paul WilsonSam Nogajski,Rod Tucker,Gerard Abood,David Boon,https://www.espncricinfo.com/series/india-in-a...,Daniel Sams,NaN,NaN
2,"Australia, elected to field first",India tour of Australia,2020/21,Yuzvendra Chahal,NaN,India led the 3-match series 1-0,T20I no. 1114,"19.10 start, First Session 19.10-20.40, Interv...",04 December 2020 - night (20-over match),Rod TuckerShawn Craig,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,T Natarajan,"Substitute: Yuzvendra Chahal in, Ravindra Jade...",NaN
3,"India, elected to bat first",India tour of New Zealand,2019/20,Jasprit Bumrah,KL Rahul,India won the 5-match series 5-0,T20I no. 1037,"20.00 start, First Session 20.00-21.30, Interv...",2 February 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Damian Morrow,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
4,"New Zealand, elected to field first",India tour of New Zealand,2019/20,Shardul Thakur,NaN,India led the 5-match series 4-0,T20I no. 1036,"20.00 start, First Session 20.00-21.30, Interv...",31 January 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Garth Stirrat,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Pakistan, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2658,NaN,9 June 2024 - day (20-over match),Richard IllingworthDRSRod TuckerDRS,Chris Gaffaney,Sharfuddoula,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Pakistan 0"
77,"India, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2639,NaN,5 June 2024 - day (20-over match),Alex WharfDRSChris GaffaneyDRS,Richard Kettleborough,Chris Brown,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Ireland 0"
78,"India, elected to bat first",Afghanistan tour of India,2023/24,Rohit Sharma,Shivam Dube,India won the 3-match series 3-0,T20I no. 2435,"19.00 start, First Session 19.00-20.30, Interv...",17 January 2024 - night (20-over match),Jayaraman MadanagopalDRSVirender SharmaDRS,Rohan Pandit,Mohit Krishnadas,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,Mohammad Saleem,NaN,NaN
79,"India, elected to field first",Afghanistan tour of India,2023/24,Axar Patel,NaN,India led the 3-match series 2-0,T20I no. 2431,"19.00 start, First Session 19.00-20.30, Interv...",14 January 2024 - night (20-over match),Rohan PanditDRSVirender SharmaDRS,Jayaraman Madanagopal,Saiyed Khalid,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,NaN,NaN,NaN


# Renaming the columns

In [37]:
new_column_names = {
    'Toss': 'Toss_Winner',
    'Series': 'Series_Name',
    'Season': 'Season_Year',
    'Player Of The Match': 'Player_Of_Match',
    'Player Of The Series': 'Player_Of_Series',
    'Series result': 'Series_Result',
    'Match number': 'MatchId',
    'Hours of play (local time)': 'Match_Hours_Local',
    'Match days': 'Match_Days',
    'Umpires': 'Umpires_List',
    'TV Umpire': 'TV_Umpire',
    'Reserve Umpire': 'Reserve_Umpire',
    'Match Referee': 'Match_Referee',
    'URL': 'Match_URL',
    'T20I debut': 'T20I_Debut',
    'IND Player Replacement': 'IND_Player_Replacement',
    'Points': 'Match_Points'
}

In [38]:
MatchDetails.rename(columns=new_column_names, inplace=True)


In [39]:
MatchDetails

,Toss_Winner,Series_Name,Season_Year,Player_Of_Match,Player_Of_Series,Series_Result,MatchId,Match_Hours_Local,Match_Days,Umpires_List,TV_Umpire,Reserve_Umpire,Match_Referee,Match_URL,T20I_Debut,IND_Player_Replacement,Match_Points
0,"India, elected to field first",India tour of Australia,2020/21,Mitchell Swepson,Hardik Pandya,India won the 3-match series 2-1,T20I no. 1116,"19.10 start, First Session 19.10-20.40, Interv...",08 December 2020 - night (20-over match),Gerard AboodRod Tucker,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,NaN,NaN,NaN
1,"India, elected to field first",India tour of Australia,2020/21,Hardik Pandya,NaN,India led the 3-match series 2-0,T20I no. 1115,"19.10 start, First Session 19.10-20.40, Interv...",06 December 2020 - night (20-over match),Paul WilsonSam Nogajski,Rod Tucker,Gerard Abood,David Boon,https://www.espncricinfo.com/series/india-in-a...,Daniel Sams,NaN,NaN
2,"Australia, elected to field first",India tour of Australia,2020/21,Yuzvendra Chahal,NaN,India led the 3-match series 1-0,T20I no. 1114,"19.10 start, First Session 19.10-20.40, Interv...",04 December 2020 - night (20-over match),Rod TuckerShawn Craig,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,T Natarajan,"Substitute: Yuzvendra Chahal in, Ravindra Jade...",NaN
3,"India, elected to bat first",India tour of New Zealand,2019/20,Jasprit Bumrah,KL Rahul,India won the 5-match series 5-0,T20I no. 1037,"20.00 start, First Session 20.00-21.30, Interv...",2 February 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Damian Morrow,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
4,"New Zealand, elected to field first",India tour of New Zealand,2019/20,Shardul Thakur,NaN,India led the 5-match series 4-0,T20I no. 1036,"20.00 start, First Session 20.00-21.30, Interv...",31 January 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Garth Stirrat,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Pakistan, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2658,NaN,9 June 2024 - day (20-over match),Richard IllingworthDRSRod TuckerDRS,Chris Gaffaney,Sharfuddoula,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Pakistan 0"
77,"India, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,T20I no. 2639,NaN,5 June 2024 - day (20-over match),Alex WharfDRSChris GaffaneyDRS,Richard Kettleborough,Chris Brown,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Ireland 0"
78,"India, elected to bat first",Afghanistan tour of India,2023/24,Rohit Sharma,Shivam Dube,India won the 3-match series 3-0,T20I no. 2435,"19.00 start, First Session 19.00-20.30, Interv...",17 January 2024 - night (20-over match),Jayaraman MadanagopalDRSVirender SharmaDRS,Rohan Pandit,Mohit Krishnadas,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,Mohammad Saleem,NaN,NaN
79,"India, elected to field first",Afghanistan tour of India,2023/24,Axar Patel,NaN,India led the 3-match series 2-0,T20I no. 2431,"19.00 start, First Session 19.00-20.30, Interv...",14 January 2024 - night (20-over match),Rohan PanditDRSVirender SharmaDRS,Jayaraman Madanagopal,Saiyed Khalid,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,NaN,NaN,NaN


In [41]:
MatchDetails['MatchId'] = MatchDetails['MatchId'].apply(lambda x: int(x.split()[-1]))

In [42]:
MatchDetails

,Toss_Winner,Series_Name,Season_Year,Player_Of_Match,Player_Of_Series,Series_Result,MatchId,Match_Hours_Local,Match_Days,Umpires_List,TV_Umpire,Reserve_Umpire,Match_Referee,Match_URL,T20I_Debut,IND_Player_Replacement,Match_Points
0,"India, elected to field first",India tour of Australia,2020/21,Mitchell Swepson,Hardik Pandya,India won the 3-match series 2-1,1116,"19.10 start, First Session 19.10-20.40, Interv...",08 December 2020 - night (20-over match),Gerard AboodRod Tucker,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,NaN,NaN,NaN
1,"India, elected to field first",India tour of Australia,2020/21,Hardik Pandya,NaN,India led the 3-match series 2-0,1115,"19.10 start, First Session 19.10-20.40, Interv...",06 December 2020 - night (20-over match),Paul WilsonSam Nogajski,Rod Tucker,Gerard Abood,David Boon,https://www.espncricinfo.com/series/india-in-a...,Daniel Sams,NaN,NaN
2,"Australia, elected to field first",India tour of Australia,2020/21,Yuzvendra Chahal,NaN,India led the 3-match series 1-0,1114,"19.10 start, First Session 19.10-20.40, Interv...",04 December 2020 - night (20-over match),Rod TuckerShawn Craig,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,T Natarajan,"Substitute: Yuzvendra Chahal in, Ravindra Jade...",NaN
3,"India, elected to bat first",India tour of New Zealand,2019/20,Jasprit Bumrah,KL Rahul,India won the 5-match series 5-0,1037,"20.00 start, First Session 20.00-21.30, Interv...",2 February 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Damian Morrow,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
4,"New Zealand, elected to field first",India tour of New Zealand,2019/20,Shardul Thakur,NaN,India led the 5-match series 4-0,1036,"20.00 start, First Session 20.00-21.30, Interv...",31 January 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Garth Stirrat,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Pakistan, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,2658,NaN,9 June 2024 - day (20-over match),Richard IllingworthDRSRod TuckerDRS,Chris Gaffaney,Sharfuddoula,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Pakistan 0"
77,"India, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,2639,NaN,5 June 2024 - day (20-over match),Alex WharfDRSChris GaffaneyDRS,Richard Kettleborough,Chris Brown,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Ireland 0"
78,"India, elected to bat first",Afghanistan tour of India,2023/24,Rohit Sharma,Shivam Dube,India won the 3-match series 3-0,2435,"19.00 start, First Session 19.00-20.30, Interv...",17 January 2024 - night (20-over match),Jayaraman MadanagopalDRSVirender SharmaDRS,Rohan Pandit,Mohit Krishnadas,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,Mohammad Saleem,NaN,NaN
79,"India, elected to field first",Afghanistan tour of India,2023/24,Axar Patel,NaN,India led the 3-match series 2-0,2431,"19.00 start, First Session 19.00-20.30, Interv...",14 January 2024 - night (20-over match),Rohan PanditDRSVirender SharmaDRS,Jayaraman Madanagopal,Saiyed Khalid,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,NaN,NaN,NaN


In [46]:
pdf = pd.read_csv("C:\\Users\\Harshita\\Documents\\BigDataFiles\\CSV_files\\new stuff\\PlayeriInfo.csv")

# Assigning PlayerIDS 

In [50]:
# Merge dataframes on the 'player_name' and 'Player_Of_Match' columns
merged_df =MatchDetails.merge(pdf[['PlayerID', 'player_name']],
                                   left_on='Player_Of_Match',
                                   right_on='player_name',
                                   how='left')
#renaming the playerID
merged_df = merged_df.rename(columns={'PlayerID': 'pom_id'})
merged_df = merged_df.drop(columns=['player_name'])


In [47]:
pdf

,PlayerID,player_name,country,date_of_birth,started_career,matches,innings,runs,hundreds,highest_score,batting_average,wickets,four_wicket_hauls,bowling_average,economy_rate,debuted,retired,state,ImageURL
0,1,Afsar Zazai,AFGHANISTAN,1993-08-10,2013-,8.0,8.0,141.0,0.0,48,17.63,0.0,0.0,-,-,2013,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
1,14,Hashmatullah Shahidi,AFGHANISTAN,1994-11-04,2013-,6.0,4.0,48.0,0.0,36,24.00,0.0,0.0,-,-,2013,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
2,15,Hazratullah Zazai,AFGHANISTAN,1998-03-23,2016-,44.0,44.0,1140.0,1.0,162,27.80,0.0,0.0,-,-,2016,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
3,16,Ibrahim Zadran,AFGHANISTAN,2001-12-12,2019-,44.0,44.0,1105.0,0.0,72,29.08,0.0,0.0,-,-,2019,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
4,17,Ihsanullah,AFGHANISTAN,1997-12-28,2022-,1.0,1.0,20.0,0.0,20,20.00,0.0,0.0,-,-,2022,0,active,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,1133,Brian Vitori,ZIMBABWE,1990-02-22,2012-2016,11.0,2.0,8.0,0.0,7,0.00,4.0,0.0,80.75,8.14,2012,2016,inactive,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
1121,1134,Malcolm Waller,ZIMBABWE,1984-09-28,2011-2018,32.0,31.0,613.0,0.0,68,26.65,0.0,0.0,-,8.40,2011,2018,inactive,http://static.cricbuzz.com/a/img/v1/50x50/i1/c...
1122,1135,Nicholas Welch,ZIMBABWE,1998-02-05,2023-,7.0,7.0,97.0,0.0,25,13.86,0.0,0.0,-,-,2023,0,active,None
1123,1136,Sean Williams,ZIMBABWE,1986-09-26,2006-,81.0,80.0,1691.0,0.0,66,23.49,48.0,0.0,28.63,6.94,2006,0,active,None


In [55]:
merged_df['pom_id']

0     156.0
1     418.0
2     372.0
3     371.0
4     459.0
      ...  
76    371.0
77    371.0
78    450.0
79    423.0
80    381.0
Name: pom_id, Length: 81, dtype: float64

In [56]:
merged_df['pom_id'] = merged_df['pom_id'].astype('Int64')


In [62]:
merged_df

,Toss_Winner,Series_Name,Season_Year,Player_Of_Match,Player_Of_Series,Series_Result,MatchId,Match_Hours_Local,Match_Days,Umpires_List,TV_Umpire,Reserve_Umpire,Match_Referee,Match_URL,T20I_Debut,IND_Player_Replacement,Match_Points,pom_id
0,"India, elected to field first",India tour of Australia,2020/21,Mitchell Swepson,Hardik Pandya,India won the 3-match series 2-1,1116,"19.10 start, First Session 19.10-20.40, Interv...",08 December 2020 - night (20-over match),Gerard AboodRod Tucker,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,NaN,NaN,NaN,156
1,"India, elected to field first",India tour of Australia,2020/21,Hardik Pandya,NaN,India led the 3-match series 2-0,1115,"19.10 start, First Session 19.10-20.40, Interv...",06 December 2020 - night (20-over match),Paul WilsonSam Nogajski,Rod Tucker,Gerard Abood,David Boon,https://www.espncricinfo.com/series/india-in-a...,Daniel Sams,NaN,NaN,418
2,"Australia, elected to field first",India tour of Australia,2020/21,Yuzvendra Chahal,NaN,India led the 3-match series 1-0,1114,"19.10 start, First Session 19.10-20.40, Interv...",04 December 2020 - night (20-over match),Rod TuckerShawn Craig,Paul Wilson,Sam Nogajski,David Boon,https://www.espncricinfo.com/series/india-in-a...,T Natarajan,"Substitute: Yuzvendra Chahal in, Ravindra Jade...",NaN,372
3,"India, elected to bat first",India tour of New Zealand,2019/20,Jasprit Bumrah,KL Rahul,India won the 5-match series 5-0,1037,"20.00 start, First Session 20.00-21.30, Interv...",2 February 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Damian Morrow,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN,371
4,"New Zealand, elected to field first",India tour of New Zealand,2019/20,Shardul Thakur,NaN,India led the 5-match series 4-0,1036,"20.00 start, First Session 20.00-21.30, Interv...",31 January 2020 - night (20-over match),Chris BrownShaun Haig,Ashley Mehrotra,Garth Stirrat,Chris Broad,https://www.espncricinfo.com/series/india-in-n...,NaN,NaN,NaN,459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"Pakistan, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,2658,NaN,9 June 2024 - day (20-over match),Richard IllingworthDRSRod TuckerDRS,Chris Gaffaney,Sharfuddoula,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Pakistan 0",371
77,"India, elected to field first",ICC Men's T20 World Cup,2024,Jasprit Bumrah,NaN,NaN,2639,NaN,5 June 2024 - day (20-over match),Alex WharfDRSChris GaffaneyDRS,Richard Kettleborough,Chris Brown,David Boon,https://www.espncricinfo.com/series/icc-men-s-...,NaN,NaN,"India 2, Ireland 0",371
78,"India, elected to bat first",Afghanistan tour of India,2023/24,Rohit Sharma,Shivam Dube,India won the 3-match series 3-0,2435,"19.00 start, First Session 19.00-20.30, Interv...",17 January 2024 - night (20-over match),Jayaraman MadanagopalDRSVirender SharmaDRS,Rohan Pandit,Mohit Krishnadas,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,Mohammad Saleem,NaN,NaN,450
79,"India, elected to field first",Afghanistan tour of India,2023/24,Axar Patel,NaN,India led the 3-match series 2-0,2431,"19.00 start, First Session 19.00-20.30, Interv...",14 January 2024 - night (20-over match),Rohan PanditDRSVirender SharmaDRS,Jayaraman Madanagopal,Saiyed Khalid,Javagal Srinath,https://www.espncricinfo.com/series/afghanista...,NaN,NaN,NaN,423


In [61]:
csv_file_path = 'MatchDetails.csv'
merged_df.to_csv(csv_file_path, index=False)